# This graph learning treats every graph (CP) to be an episodic memory of the agent.

## Turn the raw data into RDF-data

I also did ontology engineering, which is saved at [`ontology-humemai.ttl`](./ontology-humemai.ttl)


In [1]:
def is_list_of_lists_empty(list_of_lists):
    return all(not sublist for sublist in list_of_lists)

import json
import logging
from tqdm.auto import tqdm
from rdflib import URIRef, Namespace
from humemai.rdflib import Humemai
from rdflib import RDF, XSD, BNode, Graph, Literal, Namespace, URIRef

with open("./raw-data.json") as f:
    data = json.load(f)


# Turn off debug logging
logging.getLogger("humemai").setLevel(logging.WARNING)

humemai = Namespace("https://humem.ai/ontology#")
co_learning = Namespace("https://co_learning#")

memory = Humemai()
memory.graph.bind("co_learning", co_learning)



/home/tk/.virtualenvs/humemai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Treat all the CPs as events and save them as episodic memories

In [2]:

for data_point in tqdm(data):


    # declare an event
    event = URIRef(co_learning + f"CP{str(data_point['cp_num']).zfill(3)}")
    event_properties = {
        co_learning.hasCPNum: Literal(data_point["cp_num"]),
        co_learning.hasParticipantNumber: Literal(data_point["participant"]),
        co_learning.hasCPLabel: Literal(data_point["cp_name"]),
        co_learning.hasTicksLasted: Literal(data_point["ticks_lasted"]),
        co_learning.hasRoundNumber: Literal(data_point["round_num"]),
        co_learning.hasTimeStammp: Literal(data_point["timestamp"]),
        co_learning.hasUnixTimeStamp: Literal(data_point["unix_timestamp"]),
        co_learning.hasRemainingTime: Literal(data_point["remaining_time"]),
        co_learning.hasRemainingRocks: Literal(data_point["remaining_rocks"]),
        co_learning.hasVictimHarm: Literal(data_point["victim_harm"]),
    }

    # declare entities
    robot = co_learning.USAR_bot

    # add memories
    location = Literal("USAR")
    memory.add_episodic_memory(
        triples=[(robot, RDF.type, co_learning.Robot)],
        qualifiers={
            humemai.eventTime: Literal(data_point["timestamp"], datatype=XSD.dateTime),
            humemai.location: location,
            humemai.event: event,
        },
        event_properties=event_properties,
    )

    participant = URIRef(co_learning + f"P{data_point['participant']}")
    memory.add_episodic_memory(
        triples=[(participant, RDF.type, co_learning.Participant)],
        qualifiers={
            humemai.eventTime: Literal(data_point["timestamp"], datatype=XSD.dateTime),
            humemai.location: location,
            humemai.event: event,
        },
        event_properties=event_properties,
    )

    if not is_list_of_lists_empty(data_point["situation"]):

        situation_ = [
            dict(t)
            for t in {
                tuple(d.items())
                for d in [s_ for s in data_point["situation"] for s_ in s]
            }
        ]

        suffix = str(data_point["cp_num"]).zfill(3)
        situation = URIRef(co_learning + f"Situation{suffix}")

        memory.add_episodic_memory(
            triples=[(situation, RDF.type, co_learning.Situation)],
            qualifiers={
                humemai.eventTime: Literal(
                    data_point["timestamp"], datatype=XSD.dateTime
                ),
                humemai.location: location,
                humemai.event: event,
            },
            event_properties=event_properties,
        )

        for dict_ in situation_:
            assert dict_.keys() == {"type", "content"}
            if dict_["type"] == "actor":
                relation = co_learning.hasActor
            elif dict_["type"] == "object":
                relation = co_learning.hasObject
            elif dict_["type"] == "location":
                relation = co_learning.hasLocation
            elif dict_["type"] == "action":
                relation = co_learning.hasAction
            else:
                raise ValueError(f"Unknown type: {dict_['type']}")

            memory.add_episodic_memory(
                triples=[(situation, relation, Literal(dict_["content"]))],
                qualifiers={
                    humemai.eventTime: Literal(
                        data_point["timestamp"], datatype=XSD.dateTime
                    ),
                    humemai.location: location,
                    humemai.event: event,
                },
                event_properties=event_properties,
            )

    human_action_list = []
    for idx, action_ in enumerate(data_point["HumanAction"]):
        if action_:
            suffix = str(data_point["cp_num"]).zfill(3) + "_" + str(idx).zfill(3)
            human_action = URIRef(co_learning + f"HumanAction{suffix}")
            human_action_list.append(human_action)

            memory.add_episodic_memory(
                triples=[(human_action, RDF.type, co_learning.HumanAction)],
                qualifiers={
                    humemai.eventTime: Literal(
                        data_point["timestamp"], datatype=XSD.dateTime
                    ),
                    humemai.location: location,
                    humemai.event: event,
                },
                event_properties=event_properties,
            )

            for dict_ in action_:
                assert dict_.keys() == {"type", "content"}
                if dict_["type"] == "actor":
                    relation = co_learning.hasActor
                elif dict_["type"] == "object":
                    relation = co_learning.hasObject
                elif dict_["type"] == "location":
                    relation = co_learning.hasLocation
                elif dict_["type"] == "action":
                    relation = co_learning.hasAction
                else:
                    raise ValueError(f"Unknown type: {dict_['type']}")

                memory.add_episodic_memory(
                    triples=[(human_action, relation, Literal(dict_["content"]))],
                    qualifiers={
                        humemai.eventTime: Literal(
                            data_point["timestamp"], datatype=XSD.dateTime
                        ),
                        humemai.location: location,
                        humemai.event: event,
                    },
                    event_properties=event_properties,
                )

    robot_action_list = []
    for idx, action_ in enumerate(data_point["RobotAction"]):
        if action_:
            suffix = str(data_point["cp_num"]).zfill(3) + "_" + str(idx).zfill(3)
            robot_action = URIRef(co_learning + f"RobotAction{suffix}")
            robot_action_list.append(robot_action)

            memory.add_episodic_memory(
                triples=[(robot_action, RDF.type, co_learning.RobotAction)],
                qualifiers={
                    humemai.eventTime: Literal(
                        data_point["timestamp"], datatype=XSD.dateTime
                    ),
                    humemai.location: location,
                    humemai.event: event,
                },
                event_properties=event_properties,
            )

            for dict_ in action_:
                assert dict_.keys() == {"type", "content"}
                if dict_["type"] == "actor":
                    relation = co_learning.hasActor
                elif dict_["type"] == "object":
                    relation = co_learning.hasObject
                elif dict_["type"] == "location":
                    relation = co_learning.hasLocation
                elif dict_["type"] == "action":
                    relation = co_learning.hasAction
                else:
                    raise ValueError(f"Unknown type: {dict_['type']}")

                memory.add_episodic_memory(
                    triples=[(robot_action, relation, Literal(dict_["content"]))],
                    qualifiers={
                        humemai.eventTime: Literal(
                            data_point["timestamp"], datatype=XSD.dateTime
                        ),
                        humemai.location: location,
                        humemai.event: event,
                    },
                    event_properties=event_properties,
                )

    # add edges
    for idx, action in enumerate(human_action_list):

        if not is_list_of_lists_empty(data_point["situation"]):
            memory.add_episodic_memory(
                triples=[(action, co_learning.occuredIn, situation)],
                qualifiers={
                    humemai.eventTime: Literal(
                        data_point["timestamp"], datatype=XSD.dateTime
                    ),
                    humemai.location: location,
                    humemai.event: event,
                },
                event_properties=event_properties,
            )

        if idx == 0:
            memory.add_episodic_memory(
                triples=[(participant, co_learning.hasHumanAction, action)],
                qualifiers={
                    humemai.eventTime: Literal(
                        data_point["timestamp"], datatype=XSD.dateTime
                    ),
                    humemai.location: location,
                    humemai.event: event,
                },
                event_properties=event_properties,
            )
        else:
            memory.add_episodic_memory(
                triples=[
                    (human_action_list[idx - 1], co_learning.hasNextHumanAction, action)
                ],
                qualifiers={
                    humemai.eventTime: Literal(
                        data_point["timestamp"], datatype=XSD.dateTime
                    ),
                    humemai.location: location,
                    humemai.event: event,
                },
                event_properties=event_properties,
            )

    for idx, action in enumerate(robot_action_list):


        if not is_list_of_lists_empty(data_point["situation"]):
            memory.add_episodic_memory(
                triples=[(action, co_learning.occuredIn, situation)],
                qualifiers={
                    humemai.eventTime: Literal(
                        data_point["timestamp"], datatype=XSD.dateTime
                    ),
                    humemai.location: location,
                    humemai.event: event,
                },
                event_properties=event_properties,
            )        

        if idx == 0:
            memory.add_episodic_memory(
                triples=[(robot, co_learning.hasRobotAction, action)],
                qualifiers={
                    humemai.eventTime: Literal(
                        data_point["timestamp"], datatype=XSD.dateTime
                    ),
                    humemai.location: location,
                    humemai.event: event,
                },
                event_properties=event_properties,
            )
        else:
            memory.add_episodic_memory(
                triples=[
                    (robot_action_list[idx - 1], co_learning.hasNextRobotAction, action)
                ],
                qualifiers={
                    humemai.eventTime: Literal(
                        data_point["timestamp"], datatype=XSD.dateTime
                    ),
                    humemai.location: location,
                    humemai.event: event,
                },
                event_properties=event_properties,
            )

100%|██████████| 211/211 [00:00<00:00, 323.95it/s]


In [3]:
from rdflib.namespace import RDF

# Collect all subjects that are part of an rdf:Statement
reified_subjects = set()

for s, p, o in memory.graph:
    if o == RDF.Statement:
        reified_subjects.add(s)

# Loop through the graph and filter out the rdf:Statement and its related triples
for s, p, o in memory.graph:
    if s not in reified_subjects and p not in [RDF.subject, RDF.predicate, RDF.object]:
        print(s, p, o)


https://co_learning#Situation097 https://co_learning#hasLocation <Left> side of field
https://co_learning#CP114 https://co_learning#hasParticipantNumber 4091
https://co_learning#Situation177 https://co_learning#hasObject Large rock
https://co_learning#RobotAction111_000 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://co_learning#RobotAction
https://co_learning#USAR_bot https://co_learning#hasRobotAction https://co_learning#RobotAction030_000
https://co_learning#CP038 https://co_learning#hasRoundNumber 7
https://co_learning#CP004 https://co_learning#hasTimeStammp 2024-08-29T15:36:08
https://co_learning#CP109 https://co_learning#hasUnixTimeStamp 1720619625
https://co_learning#Situation118 https://co_learning#hasLocation Top of rock pile
https://co_learning#Situation163 https://co_learning#hasLocation <Left> side of rock pile
https://co_learning#CP147 https://co_learning#hasRoundNumber 4
https://co_learning#Situation100 https://co_learning#hasObject Small rock
https://co_learning#